In [46]:
from dotenv import load_dotenv
load_dotenv() 

import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [47]:
from src.inference import get_feature_store

In [48]:
feature_store = get_feature_store()

2025-05-10 06:52:52,859 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 06:52:52,881 INFO: Initializing external client
2025-05-10 06:52:52,882 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 06:52:53,492 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1225934


In [49]:
from datetime import datetime, timedelta
import pandas as pd  


current_date = pd.Timestamp.now()
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
# Ensure fetch_data_from and fetch_data_to are timezone-aware to match ts_data.pickup_hour
fetch_data_from = fetch_data_from.tz_localize(None)
fetch_data_to = fetch_data_to.tz_localize(None)
ts_data.pickup_hour = ts_data.pickup_hour.dt.tz_localize(None)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-05-10 06:52:55,508 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 06:52:55,512 INFO: Initializing external client
2025-05-10 06:52:55,512 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 06:52:56,159 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1225934
Fetching data from 2025-04-11 06:52:55.506862 to 2025-05-10 05:52:55.506862
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.84s) 


In [50]:
ts_data.sort_values(["station_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,station_id,rides
0,2025-04-12 06:00:00,HB101,2
1,2025-04-12 12:00:00,HB101,16
2,2025-04-12 18:00:00,HB101,18
3,2025-04-13 00:00:00,HB101,10
4,2025-04-13 06:00:00,HB101,13
...,...,...,...
331,2025-05-09 00:00:00,HB305,1
332,2025-05-09 06:00:00,HB305,11
333,2025-05-09 12:00:00,HB305,28
334,2025-05-09 18:00:00,HB305,3


In [51]:
ts_data

,pickup_hour,station_id,rides
0,2025-05-02 12:00:00,HB105,32
1,2025-04-25 00:00:00,HB105,4
2,2025-04-21 18:00:00,HB101,31
3,2025-04-19 06:00:00,HB101,2
4,2025-05-09 06:00:00,HB101,7
...,...,...,...
331,2025-04-17 12:00:00,HB105,20
332,2025-05-08 18:00:00,HB101,23
333,2025-05-02 06:00:00,HB305,19
334,2025-05-07 06:00:00,HB105,17


In [55]:
from src.data_utils import transform_ts_data_info_features_and_target

features , _ = transform_ts_data_info_features_and_target(ts_data, window_size=28, step_size=23)

In [56]:
features

,rides_t-28,rides_t-27,rides_t-26,rides_t-25,rides_t-24,rides_t-23,rides_t-22,rides_t-21,rides_t-20,rides_t-19,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_hour,station_id
0,32,4,27,27,8,2,32,1,15,10,...,8,10,8,28,0,43,37,28,2025-05-10 00:00:00,HB105
1,28,0,43,37,28,3,1,9,6,7,...,15,30,0,35,21,25,11,8,2025-04-30 00:00:00,HB105
2,35,21,25,11,8,1,31,39,1,15,...,10,48,18,38,19,32,34,31,2025-04-24 00:00:00,HB105
3,38,19,32,34,31,1,2,34,20,17,...,19,8,6,4,20,12,42,1,2025-05-03 12:00:00,HB105
4,31,2,7,44,12,10,2,23,0,6,...,6,16,12,31,1,12,27,0,2025-05-05 12:00:00,HB101
5,31,1,12,27,0,41,1,16,50,15,...,0,1,0,18,27,9,38,13,2025-04-25 06:00:00,HB101
6,18,27,9,38,13,8,33,8,13,1,...,41,0,16,55,9,36,25,12,2025-04-19 00:00:00,HB101
7,55,9,36,25,12,0,0,33,32,27,...,39,13,10,21,19,31,0,0,2025-04-15 00:00:00,HB101
8,14,10,24,11,1,3,4,17,7,26,...,3,5,15,4,5,8,6,4,2025-04-18 18:00:00,HB305
9,4,5,8,6,4,9,11,20,23,15,...,2,10,14,9,12,21,11,14,2025-04-20 00:00:00,HB305
